In [108]:
import pandas as pd
import numpy as np

In [110]:
#Order Report
order_report= pd.read_excel('Company X - Order Report.xlsx')

#SKU Master
sku_master = pd.read_excel('Company X - SKU Master.xlsx')

#Pincode Zones
pincode_zones = pd.read_excel('Company X - Pincode Zones.xlsx')

#Courier Invoice
courier_invoice = pd.read_excel('Courier Company - Invoice.xlsx')

#Courier Rates
courier_rates = pd.read_excel('Courier Company - Rates (1).xlsx')

In [112]:
#marging order report with SKU Master 
weight=pd.merge(order_report,sku_master)

In [114]:
weight

,ExternOrderNo,SKU,Order Qty,Weight
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150
2,2001827036,8904223819109,1.0,100
3,2001827036,8904223818430,1.0,165
4,2001827036,8904223819277,1.0,350
...,...,...,...,...
396,2001806229,8904223818942,1.0,133
397,2001806229,8904223818850,1.0,240
398,2001806226,8904223818850,2.0,240
399,2001806210,8904223816214,1.0,120


In [116]:
#checking null for cross check 
weight.isnull().sum()

ExternOrderNo    0
SKU              0
Order Qty        0
Weight           0
dtype: int64

In [118]:
#adding coloum
weight['total weight']=weight['Order Qty']*weight['Weight']
#weight per order 
weight_per_order=weight.groupby('ExternOrderNo')['total weight'].sum().reset_index()

In [120]:
#checking
weight_per_order

,ExternOrderNo,total weight
0,2001806210,220.0
1,2001806226,480.0
2,2001806229,500.0
3,2001806232,1302.0
4,2001806233,245.0
...,...,...
119,2001821995,477.0
120,2001822466,1376.0
121,2001823564,672.0
122,2001825261,1557.0


In [122]:
#Converting weight grams to kilograms 
weight_per_order['Total Weight_KG']=weight_per_order['total weight']/1000

In [124]:
#checking
weight_per_order

,ExternOrderNo,total weight,Total Weight_KG
0,2001806210,220.0,0.220
1,2001806226,480.0,0.480
2,2001806229,500.0,0.500
3,2001806232,1302.0,1.302
4,2001806233,245.0,0.245
...,...,...,...
119,2001821995,477.0,0.477
120,2001822466,1376.0,1.376
121,2001823564,672.0,0.672
122,2001825261,1557.0,1.557


In [126]:
#checking max total weight for slab 
weight_per_order.max()

ExternOrderNo      2.001827e+09
total weight       3.080000e+03
Total Weight_KG    3.080000e+00
dtype: float64

In [128]:
#determine the weight slab
def weight_slab(weight):
    if weight<=0.5:
        return 0.5
    elif weight>0.5 and weight<=1.0:
        return 1.0
    elif weight>1.0 and weight<=1.5:
        return 1.5
    elif weight>1.5 and weight<=2.0:
        return 2.0
    elif weight>2.0 and weight<=2.5:
        return 2.5
    elif weight>2.5 and weight<=3.0:
        return 3.0
    elif weight>3.0 and weight<=3.5:
        return 3.5
    else:
        return 4.0
#applying the function to determine weight slab  
weight_per_order['Weight_slab_kg']=weight_per_order['Total Weight_KG'].apply(weight_slab)

In [130]:
#checking weight slab 
weight_per_order

,ExternOrderNo,total weight,Total Weight_KG,Weight_slab_kg
0,2001806210,220.0,0.220,0.5
1,2001806226,480.0,0.480,0.5
2,2001806229,500.0,0.500,0.5
3,2001806232,1302.0,1.302,1.5
4,2001806233,245.0,0.245,0.5
...,...,...,...,...
119,2001821995,477.0,0.477,0.5
120,2001822466,1376.0,1.376,1.5
121,2001823564,672.0,0.672,1.0
122,2001825261,1557.0,1.557,2.0


In [132]:
#Merge with pincode zones to get the delivery zone 
order_zone=pd.merge(weight_per_order,courier_invoice,left_on='ExternOrderNo', right_on='Order ID', how='left')
order_zone=pd.merge(order_zone,pincode_zones,left_on=['Warehouse Pincode', 'Customer Pincode'], right_on=['Warehouse Pincode', 'Customer Pincode'], how='left')

In [134]:
# cheacking marge
order_zone

,ExternOrderNo,total weight,Total Weight_KG,Weight_slab_kg,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone_x,Type of Shipment,Billing Amount (Rs.),Zone_y
0,2001806210,220.0,0.220,0.5,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5,b
1,2001806226,480.0,0.480,0.5,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2,d
2,2001806229,500.0,0.500,0.5,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2,d
3,2001806232,1302.0,1.302,1.5,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d
4,2001806233,245.0,0.245,0.5,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2001821995,477.0,0.477,0.5,1091121183730,2001821995,0.50,121003,342008,d,Forward charges,45.4,b
170,2001822466,1376.0,1.376,1.5,1091121305541,2001822466,1.10,121003,342301,d,Forward charges,135.0,b
171,2001823564,672.0,0.672,1.0,1091121666133,2001823564,0.70,121003,492001,d,Forward and RTO charges,172.8,d
172,2001825261,1557.0,1.557,2.0,1091121981575,2001825261,1.60,121003,517128,d,Forward and RTO charges,345.0,d


In [136]:
#checking duplicated
order_zone.duplicated().sum()

50

In [138]:
#droping duplicated and reseting index
order_zone.drop_duplicates(inplace=True)
order_zone.reset_index(drop=True,inplace=True)

In [140]:
order_zone

,ExternOrderNo,total weight,Total Weight_KG,Weight_slab_kg,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone_x,Type of Shipment,Billing Amount (Rs.),Zone_y
0,2001806210,220.0,0.220,0.5,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5,b
1,2001806226,480.0,0.480,0.5,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2,d
2,2001806229,500.0,0.500,0.5,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2,d
3,2001806232,1302.0,1.302,1.5,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d
4,2001806233,245.0,0.245,0.5,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477.0,0.477,0.5,1091121183730,2001821995,0.50,121003,342008,d,Forward charges,45.4,b
120,2001822466,1376.0,1.376,1.5,1091121305541,2001822466,1.10,121003,342301,d,Forward charges,135.0,b
121,2001823564,672.0,0.672,1.0,1091121666133,2001823564,0.70,121003,492001,d,Forward and RTO charges,172.8,d
122,2001825261,1557.0,1.557,2.0,1091121981575,2001825261,1.60,121003,517128,d,Forward and RTO charges,345.0,d


In [142]:
# Merge with Courier Rates to get the rates
order_rates = pd.merge(order_zone, courier_rates, how='cross')

In [144]:
# cheacking marge
order_rates

,ExternOrderNo,total weight,Total Weight_KG,Weight_slab_kg,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone_x,...,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,2001806210,220.0,0.220,0.5,1091117221940,2001806210,2.92,121003,140604,b,...,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5
1,2001806226,480.0,0.480,0.5,1091117222065,2001806226,0.68,121003,723146,d,...,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5
2,2001806229,500.0,0.500,0.5,1091117222080,2001806229,0.71,121003,421204,d,...,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5
3,2001806232,1302.0,1.302,1.5,1091117222124,2001806232,1.30,121003,507101,d,...,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5
4,2001806233,245.0,0.245,0.5,1091117222135,2001806233,0.78,121003,263139,b,...,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477.0,0.477,0.5,1091121183730,2001821995,0.50,121003,342008,d,...,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5
120,2001822466,1376.0,1.376,1.5,1091121305541,2001822466,1.10,121003,342301,d,...,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5
121,2001823564,672.0,0.672,1.0,1091121666133,2001823564,0.70,121003,492001,d,...,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5
122,2001825261,1557.0,1.557,2.0,1091121981575,2001825261,1.60,121003,517128,d,...,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [146]:
#EXPECTED CHARGES
def pricing(row):    
    if row['Type of Shipment']=='Forward charges':                       #FORWARD CHARGES
        if row['Zone_x'] == 'a':                                         #ZONE A
            if row['Weight_slab_kg']<=0.5:
                return row['fwd_a_fixed']
            elif row['Weight_slab_kg']>0.5 and row['Weight_slab_kg']<=1.0:
                return row['fwd_a_fixed'] + row['fwd_a_additional']
            elif row['Weight_slab_kg']>1.0 and row['Weight_slab_kg']<=1.5:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*(row['Weight_slab_kg']-0.5)
            elif row['Weight_slab_kg']>1.5 and row['Weight_slab_kg']<=2.0:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>2.0 and row['Weight_slab_kg']<=2.5:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>2.5 and row['Weight_slab_kg']<=3.0:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>3.0 and row['Weight_slab_kg']<=3.5:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>3.5 and row['Weight_slab_kg']<=4.0:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*row['Weight_slab_kg']
        if row['Zone_x'] == 'b':                                         #ZONE B
            if row['Weight_slab_kg']<=0.5:
                return row['fwd_b_fixed']
            elif row['Weight_slab_kg']>0.5 and row['Weight_slab_kg']<=1.0:
                return row['fwd_b_fixed'] + row['fwd_b_additional']
            elif row['Weight_slab_kg']>1.0 and row['Weight_slab_kg']<=1.5:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*(row['Weight_slab_kg']-0.5)
            elif row['Weight_slab_kg']>1.5 and row['Weight_slab_kg']<=2.0:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>2.0 and row['Weight_slab_kg']<=2.5:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>2.5 and row['Weight_slab_kg']<=3.0:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>3.0 and row['Weight_slab_kg']<=3.5:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>3.5 and row['Weight_slab_kg']<=4.0:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*row['Weight_slab_kg']
        if row['Zone_x'] == 'c':                                         #ZONE C
            if row['Weight_slab_kg']<=0.5:
                return row['fwd_c_fixed']
            elif row['Weight_slab_kg']>0.5 and row['Weight_slab_kg']<=1.0:
                return row['fwd_c_fixed'] + row['fwd_c_additional']
            elif row['Weight_slab_kg']>1.0 and row['Weight_slab_kg']<=1.5:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>1.5 and row['Weight_slab_kg']<=2.0:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>2.0 and row['Weight_slab_kg']<=2.5:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>2.5 and row['Weight_slab_kg']<=3.0:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>3.0 and row['Weight_slab_kg']<=3.5:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>3.5 and row['Weight_slab_kg']<=4.0:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*row['Weight_slab_kg']
        if row['Zone_x'] == 'd':                                        #ZONE D
            if row['Weight_slab_kg']<=0.5:
                return row['fwd_d_fixed']
            elif row['Weight_slab_kg']>0.5 and row['Weight_slab_kg']<=1.0:
                return row['fwd_d_fixed'] + row['fwd_d_additional']
            elif row['Weight_slab_kg']>1.0 and row['Weight_slab_kg']<=1.5:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>1.5 and row['Weight_slab_kg']<=2.0:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>2.0 and row['Weight_slab_kg']<=2.5:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>2.5 and row['Weight_slab_kg']<=3.0:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>3.0 and row['Weight_slab_kg']<=3.5:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>3.5 and row['Weight_slab_kg']<=4.0:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*row['Weight_slab_kg']
        if row['Zone_x'] == 'e':                                        #ZONE E 
            if row['Weight_slab_kg']<=0.5:
                return row['fwd_e_fixed']
            elif row['Weight_slab_kg']>0.5 and row['Weight_slab_kg']<=1.0:
                return row['fwd_e_fixed'] + row['fwd_e_additional']
            elif row['Weight_slab_kg']>1.0 and row['Weight_slab_kg']<=1.5:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>1.5 and row['Weight_slab_kg']<=2.0:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>2.0 and row['Weight_slab_kg']<=2.5:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>2.5 and row['Weight_slab_kg']<=3.0:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>3.0 and row['Weight_slab_kg']<=3.5:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>3.5 and row['Weight_slab_kg']<=4.0:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*row['Weight_slab_kg']

    
    else:                                                                #FORWARD AND RTO CHARGES
        if row['Zone_x'] == 'a':                                         #ZONE A 
            if row['Weight_slab_kg']<=0.5:
                return row['fwd_a_fixed'] + row['rto_a_fixed']
            elif row['Weight_slab_kg']>0.5 and row['Weight_slab_kg']<=1.0:
                return row['fwd_a_fixed'] + row['fwd_a_additional'] + row['rto_a_fixed'] + row['rto_a_additional']
            elif row['Weight_slab_kg']>1.0 and row['Weight_slab_kg']<=1.5:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*(row['Weight_slab_kg']-0.5) + row['rto_a_fixed'] + row['rto_a_additional']+ row['rto_a_additional']*(row['Weight_slab_kg']-0.5)
            elif row['Weight_slab_kg']>1.5 and row['Weight_slab_kg']<=2.0:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*row['Weight_slab_kg']+ row['rto_a_fixed'] + row['rto_a_additional']+ row['rto_a_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>2.0 and row['Weight_slab_kg']<=2.5:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*(row['Weight_slab_kg']-0.5)+ row['rto_a_fixed'] + row['rto_a_additional']+ row['rto_a_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>2.5 and row['Weight_slab_kg']<=3.0:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*row['Weight_slab_kg']+ row['rto_a_fixed'] + row['rto_a_additional']+ row['rto_a_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>3.0 and row['Weight_slab_kg']<=3.5:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*(row['Weight_slab_kg']+0.5)+ row['rto_a_fixed'] + row['rto_a_additional']+ row['rto_a_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>3.5 and row['Weight_slab_kg']<=4.0:
                return row['fwd_a_fixed']+row['fwd_a_additional']+row['fwd_a_additional']*row['Weight_slab_kg']+ row['rto_a_fixed'] + row['rto_a_additional']+ row['rto_a_additional']*row['Weight_slab_kg']
        if row['Zone_x'] == 'b':                                         #ZONE B  
            if row['Weight_slab_kg']<=0.5:
                return row['fwd_b_fixed']+ row['rto_b_fixed'] 
            elif row['Weight_slab_kg']>0.5 and row['Weight_slab_kg']<=1.0:
                return row['fwd_b_fixed'] + row['fwd_b_additional'] + row['rto_b_fixed'] + row['rto_b_additional']
            elif row['Weight_slab_kg']>1.0 and row['Weight_slab_kg']<=1.5:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*(row['Weight_slab_kg']-0.5)+ row['rto_b_fixed'] + row['rto_b_additional']+ row['rto_b_additional']*(row['Weight_slab_kg']-0.5)
            elif row['Weight_slab_kg']>1.5 and row['Weight_slab_kg']<=2.0:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*row['Weight_slab_kg']+ row['rto_b_fixed'] + row['rto_b_additional']+ row['rto_b_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>2.0 and row['Weight_slab_kg']<=2.5:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*(row['Weight_slab_kg']+0.5)+ row['rto_b_fixed'] + row['rto_b_additional']+ row['rto_b_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>2.5 and row['Weight_slab_kg']<=3.0:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*row['Weight_slab_kg']+ row['rto_b_fixed'] + row['rto_b_additional']+ row['rto_b_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>3.0 and row['Weight_slab_kg']<=3.5:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*(row['Weight_slab_kg']+0.5)+ row['rto_b_fixed'] + row['rto_b_additional']+ row['rto_b_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>3.5 and row['Weight_slab_kg']<=4.0:
                return row['fwd_b_fixed']+row['fwd_b_additional']+row['fwd_b_additional']*row['Weight_slab_kg']+ row['rto_b_fixed'] + row['rto_b_additional']+ row['rto_b_additional']*row['Weight_slab_kg']
        if row['Zone_x'] == 'c':                                       #ZONE C 
            if row['Weight_slab_kg']<=0.5:
                return row['fwd_c_fixed']+ row['rto_c_fixed']
            elif row['Weight_slab_kg']>0.5 and row['Weight_slab_kg']<=1.0:
                return row['fwd_c_fixed'] + row['fwd_c_additional']+ row['rto_c_fixed'] + row['rto_c_additional']
            elif row['Weight_slab_kg']>1.0 and row['Weight_slab_kg']<=1.5:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*(row['Weight_slab_kg']-0.5)+ row['rto_c_fixed'] + row['rto_c_additional']+ row['rto_c_additional']*(row['Weight_slab_kg']-0.5)
            elif row['Weight_slab_kg']>1.5 and row['Weight_slab_kg']<=2.0:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*row['Weight_slab_kg']+ row['rto_c_fixed'] + row['rto_c_additional']+ row['rto_c_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>2.0 and row['Weight_slab_kg']<=2.5:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*(row['Weight_slab_kg']+0.5)+ row['rto_c_fixed'] + row['rto_c_additional']+ row['rto_c_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>2.5 and row['Weight_slab_kg']<=3.0:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*row['Weight_slab_kg']+ row['rto_c_fixed'] + row['rto_c_additional']+ row['rto_c_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>3.0 and row['Weight_slab_kg']<=3.5:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*(row['Weight_slab_kg']+0.5)+ row['rto_c_fixed'] + row['rto_c_additional']+ row['rto_c_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>3.5 and row['Weight_slab_kg']<=4.0:
                return row['fwd_c_fixed']+row['fwd_c_additional']+row['fwd_c_additional']*row['Weight_slab_kg']+ row['rto_c_fixed'] + row['rto_c_additional']+ row['rto_c_additional']*row['Weight_slab_kg']
        if row['Zone_x'] == 'd':                                   #ZONE D 
            if row['Weight_slab_kg']<=0.5:
                return row['fwd_d_fixed']+ row['rto_d_fixed'] 
            elif row['Weight_slab_kg']>0.5 and row['Weight_slab_kg']<=1.0:
                return row['fwd_d_fixed'] + row['fwd_d_additional']+ row['rto_d_fixed'] + row['rto_d_additional']
            elif row['Weight_slab_kg']>1.0 and row['Weight_slab_kg']<=1.5:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*(row['Weight_slab_kg']-0.5)+ row['rto_d_fixed'] + row['rto_d_additional']+ row['rto_d_additional']*(row['Weight_slab_kg']-0.5)
            elif row['Weight_slab_kg']>1.5 and row['Weight_slab_kg']<=2.0:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*row['Weight_slab_kg']+ row['rto_d_fixed'] +row['rto_d_additional']+ row['rto_d_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>2.0 and row['Weight_slab_kg']<=2.5:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*(row['Weight_slab_kg']+0.5)+ row['rto_d_fixed'] + row['rto_d_additional']+ row['rto_d_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>2.5 and row['Weight_slab_kg']<=3.0:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*row['Weight_slab_kg']+ row['rto_d_fixed'] + row['rto_d_additional']+ row['rto_d_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>3.0 and row['Weight_slab_kg']<=3.5:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*(row['Weight_slab_kg']+0.5)+ row['rto_d_fixed'] + row['rto_d_additional']+ row['rto_d_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>3.5 and row['Weight_slab_kg']<=4.0:
                return row['fwd_d_fixed']+row['fwd_d_additional']+row['fwd_d_additional']*row['Weight_slab_kg']+ row['rto_d_fixed'] + row['rto_d_additional']+ row['rto_d_additional']*row['Weight_slab_kg']
        if row['Zone_x'] == 'e':                                    #ZONE E 
            if row['Weight_slab_kg']<=0.5:
                return row['fwd_e_fixed']+ row['rto_e_fixed'] 
            elif row['Weight_slab_kg']>0.5 and row['Weight_slab_kg']<=1.0:
                return row['fwd_e_fixed'] + row['fwd_e_additional']+ row['rto_e_fixed'] + row['rto_e_additional']
            elif row['Weight_slab_kg']>1.0 and row['Weight_slab_kg']<=1.5:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*(row['Weight_slab_kg']-0.5)+ row['rto_e_fixed'] + row['rto_e_additional']+ row['rto_e_additional']*(row['Weight_slab_kg']-0.5)
            elif row['Weight_slab_kg']>1.5 and row['Weight_slab_kg']<=2.0:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*row['Weight_slab_kg']+ row['rto_e_fixed'] + row['rto_e_additional']+ row['rto_e_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>2.0 and row['Weight_slab_kg']<=2.5:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*(row['Weight_slab_kg']+0.5)+ row['rto_e_fixed'] + row['rto_e_additional']+ row['rto_e_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>2.5 and row['Weight_slab_kg']<=3.0:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*row['Weight_slab_kg']+ row['rto_e_fixed'] + row['rto_e_additional']+ row['rto_e_additional']*row['Weight_slab_kg']
            elif row['Weight_slab_kg']>3.0 and row['Weight_slab_kg']<=3.5:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*(row['Weight_slab_kg']+0.5)+ row['rto_e_fixed'] + row['rto_e_additional']+ row['rto_e_additional']*(row['Weight_slab_kg']+0.5)
            elif row['Weight_slab_kg']>3.5 and row['Weight_slab_kg']<=4.0:
                return row['fwd_e_fixed']+row['fwd_e_additional']+row['fwd_e_additional']*row['Weight_slab_kg']+ row['rto_e_fixed'] + row['rto_e_additional']+ row['rto_e_additional']*row['Weight_slab_kg']
       

In [148]:
#APPLYING AND MAKING NEW COLOUMN 
order_rates['Expected Charge(Rs)']=order_rates.apply(pricing, axis=1)

In [150]:
#CHECKING
order_rates

,ExternOrderNo,total weight,Total Weight_KG,Weight_slab_kg,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone_x,...,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional,Expected Charge(Rs)
0,2001806210,220.0,0.220,0.5,1091117221940,2001806210,2.92,121003,140604,b,...,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,33.0
1,2001806226,480.0,0.480,0.5,1091117222065,2001806226,0.68,121003,723146,d,...,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,45.4
2,2001806229,500.0,0.500,0.5,1091117222080,2001806229,0.71,121003,421204,d,...,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,45.4
3,2001806232,1302.0,1.302,1.5,1091117222124,2001806232,1.30,121003,507101,d,...,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,179.8
4,2001806233,245.0,0.245,0.5,1091117222135,2001806233,0.78,121003,263139,b,...,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477.0,0.477,0.5,1091121183730,2001821995,0.50,121003,342008,d,...,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,45.4
120,2001822466,1376.0,1.376,1.5,1091121305541,2001822466,1.10,121003,342301,d,...,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,179.8
121,2001823564,672.0,0.672,1.0,1091121666133,2001823564,0.70,121003,492001,d,...,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,176.3
122,2001825261,1557.0,1.557,2.0,1091121981575,2001825261,1.60,121003,517128,d,...,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,355.5


In [152]:
#DIFFERENCE BETWEEN EXPEXTED AND BILLED CHARGES
order_rates['Difference(Rs)']=order_rates['Expected Charge(Rs)']-order_rates['Billing Amount (Rs.)']

In [154]:
#CHECKING
order_rates

,ExternOrderNo,total weight,Total Weight_KG,Weight_slab_kg,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone_x,...,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional,Expected Charge(Rs),Difference(Rs)
0,2001806210,220.0,0.220,0.5,1091117221940,2001806210,2.92,121003,140604,b,...,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,33.0,-141.5
1,2001806226,480.0,0.480,0.5,1091117222065,2001806226,0.68,121003,723146,d,...,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,45.4,-44.8
2,2001806229,500.0,0.500,0.5,1091117222080,2001806229,0.71,121003,421204,d,...,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,45.4,-44.8
3,2001806232,1302.0,1.302,1.5,1091117222124,2001806232,1.30,121003,507101,d,...,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,179.8,44.8
4,2001806233,245.0,0.245,0.5,1091117222135,2001806233,0.78,121003,263139,b,...,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,33.0,-28.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,477.0,0.477,0.5,1091121183730,2001821995,0.50,121003,342008,d,...,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,45.4,0.0
120,2001822466,1376.0,1.376,1.5,1091121305541,2001822466,1.10,121003,342301,d,...,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,179.8,44.8
121,2001823564,672.0,0.672,1.0,1091121666133,2001823564,0.70,121003,492001,d,...,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,176.3,3.5
122,2001825261,1557.0,1.557,2.0,1091121981575,2001825261,1.60,121003,517128,d,...,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5,355.5,10.5


In [156]:
result= order_rates[['Order ID', 'AWB Code', 'Total Weight_KG', 'Weight_slab_kg', 'Charged Weight', 'Zone_x','Zone_y','Type of Shipment','Expected Charge(Rs)', 'Billing Amount (Rs.)','Difference(Rs)']].round(2)

# Saving the result DataFrame to an Excel file
result.to_excel('Result.xlsx', index=False)

In [158]:
result

,Order ID,AWB Code,Total Weight_KG,Weight_slab_kg,Charged Weight,Zone_x,Zone_y,Type of Shipment,Expected Charge(Rs),Billing Amount (Rs.),Difference(Rs)
0,2001806210,1091117221940,0.22,0.5,2.92,b,b,Forward charges,33.0,174.5,-141.5
1,2001806226,1091117222065,0.48,0.5,0.68,d,d,Forward charges,45.4,90.2,-44.8
2,2001806229,1091117222080,0.50,0.5,0.71,d,d,Forward charges,45.4,90.2,-44.8
3,2001806232,1091117222124,1.30,1.5,1.30,d,d,Forward charges,179.8,135.0,44.8
4,2001806233,1091117222135,0.24,0.5,0.78,b,b,Forward charges,33.0,61.3,-28.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.48,0.5,0.50,d,b,Forward charges,45.4,45.4,0.0
120,2001822466,1091121305541,1.38,1.5,1.10,d,b,Forward charges,179.8,135.0,44.8
121,2001823564,1091121666133,0.67,1.0,0.70,d,d,Forward and RTO charges,176.3,172.8,3.5
122,2001825261,1091121981575,1.56,2.0,1.60,d,d,Forward and RTO charges,355.5,345.0,10.5


In [166]:
#summary
summary=pd.DataFrame({'Summary':['Total orders where X has been correctly charged',
                                 'Total Orders where X has been overcharged',
                                 'Total Orders where X has been undercharged'],
                     'Total order':[result[result['Billing Amount (Rs.)'] == result['Expected Charge(Rs)']]['Difference(Rs)'].count(),
                                    result[result['Billing Amount (Rs.)'] > result['Expected Charge(Rs)']]['Difference(Rs)'].count(),
                                    result[result['Billing Amount (Rs.)'] < result['Expected Charge(Rs)']]['Difference(Rs)'].count()],
                     'Amount':[result[result['Billing Amount (Rs.)'] == result['Expected Charge(Rs)']]['Difference(Rs)'].sum(),
                                result[result['Billing Amount (Rs.)'] > result['Expected Charge(Rs)']]['Difference(Rs)'].sum(),
                                result[result['Billing Amount (Rs.)'] < result['Expected Charge(Rs)']]['Difference(Rs)'].sum()]}) 

In [168]:
summary

,Summary,Total order,Amount
0,Total orders where X has been correctly charged,50,0.0
1,Total Orders where X has been overcharged,51,-3140.8
2,Total Orders where X has been undercharged,23,586.6


In [170]:
# Save the summary DataFrame to an csv file
summary.to_csv('Summary.csv')